In [8]:
import pandas as pd

# Import your libraries
import pandas as pd
from pandas import Timestamp
# Start writing code
df=doordash_delivery
temp=[]
for k in df.keys():
    print(k,end=',')

# for i in range(len(df)):
#     row=df.iloc[i]
#     s=''
#     for k in row.keys():
        
#         s+=str(row[k])+','
#     print(s)



print()

In [36]:
df=pd.read_csv("input.csv")
#get the whole column at index 0
df.iloc[:,0]



0     2020-05-25 16:36:00
1     2020-05-02 16:32:00
2     2020-05-24 16:38:00
3     2020-05-17 17:34:00
4     2020-05-24 17:40:00
             ...         
91    2020-05-21 16:29:00
92    2020-05-04 00:11:00
93    2020-05-27 01:28:00
94    2020-05-23 00:27:00
95    2020-05-13 00:00:00
Name: customer_placed_order_datetime, Length: 96, dtype: object